In [1]:
import os
import sys
sys.path.append('../va_pipeline/')

In [2]:
import subprocess
import pandas as pd
import numpy as np

from calculate_accuracy import *
from sensor import exec_file


In [3]:
# Change to name and path of output files
res_width = 640
res_height = 360
model = 'yolov5n'
source = 'sparse'
dest = f'frame_diff/{source}'
framerate = 25
frame_cap = 250
save_results = True

test_dir = f'./test_results/config_testing/{dest}/'
test_name = f'diff_{source}_{model}_{res_width}_{res_height}_{framerate}fps'
test_path = test_dir + test_name

gt_path = f'./test_results/config_testing/{source}_yolov5l_ground_truth.csv'
gt = get_ground_truth_list(res_width, res_height, gt_path)

# Get preds list
# pred_path = f'{test_path}' +'_inference.csv'
pred_path = './test_results/temp.csv'
preds = get_predictions_list(res_width, res_height, pred_path)

# Calculate mAP scores
mAP = calculate_accuracy(gt, preds)

In [4]:
print(mAP)

0.07586567848920822


In [6]:
len(gt)

250

In [8]:
len(preds)

249